<a href="https://colab.research.google.com/github/Homura-san/pyspark/blob/main/TrabTopBD_op2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de Dependências

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
# instalando a findspark
!pip install -q findspark

In [7]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

In [8]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparando datasets

In [10]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.ml.feature import PCA
#from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

### Dataset dados gerais da covid 2021 por data de notificação

In [11]:
dadosCovid = sc.read.csv(
    path = "/content/drive/My Drive/dadosGerais2021.csv", 
    inferSchema = True, 
    header = True,
    sep = ',', 
    encoding = "UTF-8")

In [ ]:
type(dadosCovid)

pyspark.sql.dataframe.DataFrame

In [ ]:
dadosCovid.show()

+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+---------------------------------------------+-------------------------------------------+----------------------------------------------------------------------+--------+---------------+--------------------------+----------+
|DATA_DA_NOTIFICACAO|DATA_DE_NASCIMENTO| CLASSIFICACAO_FINAL|MUNICIPIO_DE_RESIDENCIA|RACA_COR|SEXO|E_PROFISSIONAL_DE_SAUDE|FEBRE|TOSSE|DOR_DE_GARGANTA|DISPNEIA|              OUTROS|DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS|DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO|PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA|DIABETES|IMUNOSSUPRESSAO|DOENCAS_CARDIACAS_CRONICAS|    STATUS|
+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+-------

In [43]:
dadosCovid.printSchema()

root
 |-- DATA_DA_NOTIFICACAO: string (nullable = true)
 |-- DATA_DE_NASCIMENTO: string (nullable = true)
 |-- CLASSIFICACAO_FINAL: string (nullable = true)
 |-- MUNICIPIO_DE_RESIDENCIA: string (nullable = true)
 |-- RACA_COR: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- E_PROFISSIONAL_DE_SAUDE: string (nullable = true)
 |-- FEBRE: string (nullable = true)
 |-- TOSSE: string (nullable = true)
 |-- DOR_DE_GARGANTA: string (nullable = true)
 |-- DISPNEIA: string (nullable = true)
 |-- OUTROS: string (nullable = true)
 |-- DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS: string (nullable = true)
 |-- DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO: string (nullable = true)
 |-- PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA: string (nullable = true)
 |-- DIABETES: string (nullable = true)
 |-- IMUNOSSUPRESSAO: string (nullable = true)
 |-- DOENCAS_CARDIACAS_CRONICAS: string (nullable = true)
 |-- STATUS: string (nullable = true)



In [41]:
temp1 = dadosCovid.groupby('CLASSIFICACAO_FINAL').count()
temp1.show()

+--------------------+------+
| CLASSIFICACAO_FINAL| count|
+--------------------+------+
|Confirmado por Cr...|   248|
|          Confirmado|    12|
|                null| 14297|
|Vinculo- Epidemio...|   113|
|Confirmado Clínic...|   546|
|        Laboratorial|  7789|
|Confirmado Labora...|100935|
|Síndrome Gripal N...|     8|
|             Clínico|   184|
|                   4|     6|
|Confirmado Clínic...|    13|
|          Descartado|   766|
+--------------------+------+



### Especificação dos tipos de casos notificados no ano de 2021

In [42]:
# Especificação dos tipos de casos notificados no ano de 2021
tiposDeCasos = temp1.toPandas()
tiposDeCasos.head(12)

,CLASSIFICACAO_FINAL,count
0,Confirmado por Critério Clínico,248
1,Confirmado,12
2,None,14297
3,Vinculo- Epidemiológico,113
4,Confirmado Clínico-Epidemiológico,546
5,Laboratorial,7789
6,Confirmado Laboratorial,100935
7,Síndrome Gripal Não Especificada,8
8,Clínico,184
9,4,6


In [ ]:
# Converter para csv
tiposDeCasos.to_csv('tiposDeCasos.csv')

### Filtrando casos não confirmados e mal-notificados

In [22]:
# Filtrando casos descartados e com classificação nula
casosNaoNulos = dadosCovid.filter(dadosCovid['CLASSIFICACAO_FINAL'] != 'null')
casosNaoDesc = casosNaoNulos.filter(dadosCovid.CLASSIFICACAO_FINAL != 'Descartado')

In [23]:
# Filtrando casos com classificação incerta
casosNao4 = casosNaoDesc.filter(dadosCovid.CLASSIFICACAO_FINAL != '4')

In [24]:
# FIltrando casos de síndrome gripal não especificada
casosConfirmTotais = casosNao4.filter(dadosCovid.CLASSIFICACAO_FINAL.contains('Síndrome Gripal Não Especificada') == False)
casosConfirmados = casosConfirmTotais.count()
# Imprimindo total de casos confirmados
print(casosConfirmados)

109840


In [27]:
profSaude = casosConfirmTotais.groupby('E_PROFISSIONAL_DE_SAUDE').count()

#### Criando datasets para subdivisões dos casos confirmados totais

Todos os casos confirmados entre profissionais de saúde

In [28]:
# Dado que pode ser incluído no relatório - Todos os casos confirmados entre profissionais de saúde
p1 = profSaude.toPandas()
p1.head()

,E_PROFISSIONAL_DE_SAUDE,count
0,None,3982
1,Sim,3532
2,Nao,102326


In [ ]:
# Salvar como arquivo csv
p1.to_csv("casosProfSaude.csv")

Casos confirmados totais por sexo

In [82]:
casosPorSexo = casosConfirmTotais.groupby('SEXO').count()
casosPorSexo.show()

+----+-----+
|SEXO|count|
+----+-----+
|   F|58921|
|   M|50912|
|   I|    7|
+----+-----+



In [83]:
c1 = casosPorSexo.toPandas()
c1.head()

,SEXO,count
0,F,58921
1,M,50912
2,I,7


In [84]:
# Salvar como arquivo csv
c1.to_csv("casosPorSexo.csv")

Casos confirmados por etnia

In [85]:
casosPorEtnia = casosConfirmTotais.groupby('RACA_COR').count()
casosPorEtnia.show()

+--------+-----+
|RACA_COR|count|
+--------+-----+
|Indígena|    9|
|Ignorado|43130|
|   Preta|13495|
| Amarela| 1460|
|  Branca| 9956|
|   Parda|41790|
+--------+-----+



In [86]:
c2 = casosPorEtnia.toPandas()
c2.head()

,RACA_COR,count
0,Indígena,9
1,Ignorado,43130
2,Preta,13495
3,Amarela,1460
4,Branca,9956


In [87]:
# Salvar como arquivo csv
c2.to_csv("casosPorEtnia.csv")

##### Filtrar e contabilizar casos de covid de acordo com o mês

Separando datasets por mês

In [44]:
casosMeses = []
for i in range(12):
  k = None
  if i < 9:
    k = f'2021-0{i+1}'
  else:
    k = f'2021-{i+ 1}'
  t1 = casosConfirmTotais.filter(casosConfirmTotais.DATA_DA_NOTIFICACAO.contains(f'{k}'))
  casosMeses.append(t1)

In [58]:
casosMeses[0].show()

+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+---------------------------------------------+-------------------------------------------+----------------------------------------------------------------------+--------+---------------+--------------------------+----------+
|DATA_DA_NOTIFICACAO|DATA_DE_NASCIMENTO| CLASSIFICACAO_FINAL|MUNICIPIO_DE_RESIDENCIA|RACA_COR|SEXO|E_PROFISSIONAL_DE_SAUDE|FEBRE|TOSSE|DOR_DE_GARGANTA|DISPNEIA|              OUTROS|DOENCAS_RESPIRATORIAS_CRONICAS_DESCOMPENSADAS|DOENCAS_RENAIS_CRONICAS_EM_ESTAGIO_AVANCADO|PORTADOR_DE_DOENCAS_CROMOSSOMICAS_OU_ESTADO_DE_FRAGILIDADE_IMUNOLOGICA|DIABETES|IMUNOSSUPRESSAO|DOENCAS_CARDIACAS_CRONICAS|    STATUS|
+-------------------+------------------+--------------------+-----------------------+--------+----+-----------------------+-----+-----+---------------+--------+--------------------+-------

In [78]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

data2 = []
for i in range(len(casosMeses)):
  temp = casosMeses[i].count()
  t1 = None
  
  if i < 9:
    t1 = (f"20210{i+1}", temp)
  else:
    t1 = (f"2021{i+1}", temp)

  data2.append(t1)

schema = StructType([StructField("MES", StringType(), True),
                     StructField("NUMERO_DE_CASOS", IntegerType(), True)
])

df_casosPorMes = sc.createDataFrame(data=data2, schema=schema)

In [79]:
df_casosPorMes.show()

+------+---------------+
|   MES|NUMERO_DE_CASOS|
+------+---------------+
|202101|          13694|
|202102|          20579|
|202103|          23100|
|202104|          14009|
|202105|          16525|
|202106|          12523|
|202107|           5868|
|202108|           1850|
|202109|            785|
|202110|            342|
|202111|            277|
|202112|            288|
+------+---------------+



In [80]:
casosPM = df_casosPorMes.toPandas()
casosPM.head()

,MES,NUMERO_DE_CASOS
0,202101,13694
1,202102,20579
2,202103,23100
3,202104,14009
4,202105,16525


In [ ]:
# Salvar como arquivo csv
casosPM.to_csv("casosPorMes.csv")

Revisão de tópicos dispiníveis em csv:



*   Tipos de casos totais notificados
*   Casos confirmados entre profissionais de saúde
*   Número de casos confirmados por mês
*   Casos confirmados por sexo
*   Casos confirmados por etnia



### Preparando dataset - Número de vacinações Covid 

In [12]:
vacinacaoCovid = sc.read.csv(
    path = "/content/drive/My Drive/vacinacaoPorDia.csv", 
    inferSchema = True, 
    header = True,
    sep = ',', 
    encoding = "UTF-8")

In [29]:
vacinacaoCovid.show()

+---------+-------+
|    DATAS|NUMEROS|
+---------+-------+
|19-JAN-21|   1396|
|20-JAN-21|   3918|
|21-JAN-21|   3404|
|22-JAN-21|   2687|
|23-JAN-21|   2875|
|24-JAN-21|   3242|
|25-JAN-21|   3198|
|26-JAN-21|   2244|
|27-JAN-21|   1643|
|28-JAN-21|   2874|
|29-JAN-21|   8649|
|30-JAN-21|  10202|
|31-JAN-21|   7144|
|01-FEB-21|   4154|
|02-FEB-21|   2869|
|03-FEB-21|   2782|
|04-FEB-21|   5058|
|05-FEB-21|   7966|
|06-FEB-21|   8786|
|07-FEB-21|   1357|
+---------+-------+
only showing top 20 rows



In [30]:
vacinacaoCovid.printSchema()

root
 |-- DATAS: string (nullable = true)
 |-- NUMEROS: integer (nullable = true)



In [31]:
mes = ['JAN-21', 'FEB-21', 'MAR-21', 'APR-21', 'MAY-21', 'JUN-21', 'JUL-21', 'AUG-21', 'SEP-21', 'OCT-21', 'NOV-21', 'DEC-21']
vacinacaoCovid2021 = []

In [32]:
for i in range(1, 13):
  globals()[f"mes{i}"] = vacinacaoCovid.filter(vacinacaoCovid.DATAS.contains(f"{mes[i-1]}"))
  vacinacaoCovid2021.append(globals()[f"mes{i}"])

print(vacinacaoCovid2021)

# Como eu posso somar os valores de cada linha??

[DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int], DataFrame[DATAS: string, NUMEROS: int]]


In [ ]:
# Janeiro = índice 0;        Fevereiro = índice 1; Assim por diante
vacinacaoCovid2021[0].show()

+---------+-------+
|    DATAS|NUMEROS|
+---------+-------+
|19-JAN-21|   1396|
|20-JAN-21|   3918|
|21-JAN-21|   3404|
|22-JAN-21|   2687|
|23-JAN-21|   2875|
|24-JAN-21|   3242|
|25-JAN-21|   3198|
|26-JAN-21|   2244|
|27-JAN-21|   1643|
|28-JAN-21|   2874|
|29-JAN-21|   8649|
|30-JAN-21|  10202|
|31-JAN-21|   7144|
+---------+-------+



In [33]:
for i in range(len(vacinacaoCovid2021)):
  a = None
  if i < 9:
    a = f'20210{i+1}'
  else:
    a = f'2021{i+ 1}'
  temp = vacinacaoCovid2021[i].withColumn('DATAS', F.when(vacinacaoCovid2021[i].DATAS.contains(f'{mes[i]}'), a)\
                                        .otherwise(vacinacaoCovid2021[i].DATAS))
  vacinacaoCovid2021[i] = temp
  vacinacaoCovid2021[i] = vacinacaoCovid2021[i].groupBy(F.col('DATAS')).sum('NUMEROS')
  #temp.groupBy(F.col('DATAS')).sum('NUMEROS').show()

In [34]:
vacinacaoCovidMes = None
for i in range(len(vacinacaoCovid2021) - 1):
  if i == 0:
    vacinacaoCovidMes = vacinacaoCovid2021[0].unionByName(vacinacaoCovid2021[i + 1])
  else:
    vacinacaoCovidMes = vacinacaoCovidMes.unionByName(vacinacaoCovid2021[i + 1])

vacinacaoCovidMes.show()

+------+------------+
| DATAS|sum(NUMEROS)|
+------+------------+
|202101|       53476|
|202102|      128018|
|202103|      253793|
|202104|      333229|
|202105|      360640|
|202106|      354389|
|202107|      480148|
|202108|      723756|
|202109|      591456|
|202110|      504166|
|202111|      435901|
|202112|      357592|
+------+------------+



In [ ]:
# Não executar
from pyspark.sql.types import StringType, DateType, FloatType

v2 = vacinacaoCovidMes.withColumn("DATAS", 
                                  vacinacaoCovidMes["DATAS"].cast(types.DateType()))

v2.printSchema()

In [39]:
# Convertendo para o Pandas para sakvar como csv
vacMes = vacinacaoCovidMes.toPandas()
print(vacMes)
#vacinacaoCovidMes.write.csv('vacinacaoCovidMes.csv')

     DATAS  sum(NUMEROS)
0   202101         53476
1   202102        128018
2   202103        253793
3   202104        333229
4   202105        360640
5   202106        354389
6   202107        480148
7   202108        723756
8   202109        591456
9   202110        504166
10  202111        435901
11  202112        357592


In [36]:
# Salvar no formato csv - consultar na aba arquivos
vacMes.to_csv("vacinacaoPorMes.csv")